# Estimating PoST proof duration

In [50]:
from IPython.display import display,Math,Markdown
from scipy.stats import binom, nbinom
from scipy import optimize
import math
from math import log, floor, ceil, sqrt
import random

def dm(str):
    display(Markdown(str))


The PoST proof is a long-running task whose duration is probabilistic. However, given the disk read speed and AES computation speed, we know the exactly how the duration is distributed. Moreover, as the PoST proof progresses, we get additional information, and can update both the expected time and the percentiles.

## Background

To recall, the PoST proof basically repeats the following steps until it is successful:
Let $n$ be the number of labels in the PoST data.
1. Compute $r$ k2pows for set of $m=r\cdot b$ parallel nonces, in batches of size $b$.
2. For $i$ in range ($n$):

   a.  read label $i$
   
   b.  hash label $i$ (number of hashes is $r$)
   
   c.  For every nonce, check if hashed label passes $k_1/n$ threshold. Call such a label "good"
   
   d.  Return **success**  if, for any nonce, we have found at least $k_2$  good labels.

For a rough estimate, let's assume we never stop in the middle of a pass (i.e., even if we found $k_2$ good labels we still do the whole pass).  This simplifies things, since the time for a single complete pass depends only on k2pow,  read and compute speed (this simplification means we always overestimate the time). Let's denote $K_i$ the number of hash invocations it takes to solve the $i^{th}$ k2pow, and $R$ the time it takes to do a single read/hash pass over the data for the $m$ nonces. 

R is essentially a constant that depends on the CPU/disk parameters. We can estimate $R$ by benchmarking for a small set of  $\alpha\cdot n$ labels, then dividing by $\alpha$. For a single nonce batch, $K_i$ is a random variable that is [geometrically](https://en.wikipedia.org/wiki/Geometric_distribution) distributed with parameter $1/d$, where $d$ is the expected k2pow difficulty (i.e., expected number of hash invocations).  For multiple nonce batches (i.e., $r>1$), $K_i$ is the sum of $r$ geometric variables, which is distributed as a [negative binomial](https://en.wikipedia.org/wiki/Negative_binomial_distribution), with parameters $p=1/d$ and $r$.

Denote $T$ the total time for $x$ passes over the data, including the k2pow. Then
$$ T=R\cdot x+\frac{\sum_{i=1}^{x} K_i}{\text{hashrate}} $$

For a single nonce, the number of good labels out of $n$ is distributed $B(n,k_1/n)$ --- that is, [binomially](https://en.wikipedia.org/wiki/Binomial_distribution) with parameters $n$ and $p=k_1/n$.  Since passes are independent, at the beginning of  every pass, the probability that this pass will be successful *for a specific nonce* is $q=Pr[B(n,k1/n) \ge k_2]$. We are trying $m$ nonces in parallel, and a pass will fail only if *all* of the nonces fail. Since the events are independent for every nonce, the probability that a pass succeeds is $q_m = 1-(1-q)^m$.


In [2]:
def pass_success_prob(k1: int, k2: int, m: int, n: int) -> float:
    q = binom.sf(k2-1,n,k1/n)
    qm = 1 - (1-q)**m
    return qm


## Apriori duration distribution (i.e., at the beginning of a pass)
The success of each pass is completely independent of previous successes (and also independent of $R$), so the number of passes is distributed geometrically with parameter $q_m$.  This gives us a formula for the expected time at the beginning of a pass: 
   $$E[T] = \sum_{i=1}^{1/q_m} E[R+K_i] = \sum_{i=1}^{1/q_m} (E[R]+E[K_i]) = (R+d\cdot r/\text{hashrate})/q_m \ . $$



In [32]:
def expected_pass_k2pow_time(d: int, hashrate: int, r: int) -> float:
    return d*r/hashrate

def expected_total_time_apriori(R: float, d: int, hashrate: int, r: int, qm: float) -> float:
    """ Return the total expected time to success
    :param float R:        time to read/process labels for one pass 
    :param float d:        expected number of hashes to solve k2pow for single nonce batch
    :param int   hashrate: number of k2pow hashes per time unit 
    :param int   r:        number of nonce batches
    :param float qm:       probability that a single pass succeeds."""
    
    return (R+expected_pass_k2pow_time(d, hashrate, r))/qm

### Percentile Calculation
The percentile calculation is a little more complex: the number of passes required to succeed with probability $p^*$ is $\log(1-p^*)/\log(1-q_m)$. (To see this, let $X$ be the number of failures before success, note that for every $x\ge 1$, $\Pr[X>x]=(1-q_m)^x$, then solve for $x$ such that $p^*=\Pr[X\le x] = 1-\Pr[X>x]$.)

If we assume the k2pow effort $K_i=d\cdot r$ is a constant, this gives us a simple percentile calculation: $(R+d\cdot r/\text{hashrate})\cdot \log(1-p^*)/\log(1-q_m)$.
Since $K_i$ is not constant, computing the percentile exactly is more involved. However, we can still lower-bound the time for a given percentile.

To do this, we consider separately the time for reading/processing nonces and the total k2pow time. The former is given by $R\cdot \log(1-p^*)/\log(1-q_m)$. For the latter, if we condition on the number of passes being $x$, then we need to solve $x\cdot r$ k2pows, hence the total k2pow effort $K=\sum_{i=1}^x K_i$ is distributed as a negative binomial variable with parameters $1/d$ and $x\cdot r$.

For every $\epsilon\in (0,1-p^*)$, we can find $K^*$ and $x$ such that $\Pr[K\le K^* | X\le x] \ge (1-\epsilon)$ and $\Pr[X \le x] \ge p^*/(1-\epsilon)$. Then 
    $$ \Pr[T < Rx+K^*/\text{hashrate}] \ge \Pr[X \le x \wedge K \le K^*] = \Pr[K \le K^* | X \le x ]\cdot \Pr[X \le x] \ge p^* $$
    
Thus, every $\epsilon$ gives a bound on the time $T$ for the $p^*$ percentile. To find the best bound, we do a search over $\epsilon\in (0,1-p^*)$ that minimizes $Rx+K^*/\text{hashrate}$.

In [34]:
def percentile_passes(pstar: float, qm: float) -> int:
    """ Return number of passes x such that with probability at least pstar we need x passes. """
    return ceil(log(1-pstar)/log(1-qm))

def percentile_k2pow_time(pstar: float, d: int, hashrate: int, r: int) -> float:
    return nbinom.ppf(pstar, r + 1, 1/d) / hashrate

def simple_percentile_total_time_apriori(pstar: float, R: float, d: int, hashrate: int, r: int, qm: float) -> float:
    return (R+expected_pass_k2pow_time(d, hashrate, r))*percentile_passes(pstar, qm)

# We define functions that allow the first k2pow to have a different number of batches in order to support 
# the mid-pass percentile function below.
def percentile_bound_total_time_apriori(eps: float, pstar: float, R: float, d: int, hashrate: int, r: int, qm: float, rfirst: int = -1) -> float:
    if (rfirst < 0):
        rfirst = r
    x = percentile_passes(pstar/(1-eps), qm)
    kstar_time = percentile_k2pow_time(1-eps, d, hashrate, rfirst + r*(x-1))
    return R*x + kstar_time


def percentile_opt_total_time_apriori(pstar: float, R: float, d: int, hashrate: int, r: int, qm: float, rfirst: int = -1) -> float:
    """Find p such that Pr[X \ge k]=q when X~Binom(n,p)"""
    def time_bound(eps):
        return percentile_bound_total_time_apriori(eps, pstar, R, d, hashrate, r, qm, rfirst)

    bounds=(0, 1-pstar)
        
    res = optimize.minimize_scalar(time_bound,bounds=bounds, method='bounded', options={'xatol': 2**(-64), 'maxiter': 2**20})
        
    return res.fun


## Mid-pass duration distribution
To analyze the duration distribution in the middle of a pass, we first analyze the remaining duration *in the current pass* (ignoring the total duration). We can denote this as 
$T'=K'/hashrate + R'$, where $K'$ is the remaining k2pow attempts for this pass, and $R'$ the remaining read/process time in this pass. 

### Current pass: mid-k2pow
During the k2pow phase, $R'=R$ (since we haven't started reading yet), and the only new information we receive that can affect our time estimations is solving another k2pow out of the $r$ batches we must solve. If we have $r'$ remaining k2pows to solve, then $K'$ is distributed negative-binomially with parameters $1/d$ and $r'$.  

### Current pass: post-k2pow
After completing k2pow for this pass, $K'=0$, and $R'$ depends only on the fraction of labels we've read/processed so far. After reading $\gamma\cdot n$ labels, $R'=(1-\gamma)\cdot R$.


### Total duration
To compute the total remaining duration, we separate into two cases: 
 1. the current pass is successful (hence, the last pass) and
 2. the current pass fails.

In the first case, the remaining time is only the current-pass remaining time. In the second case, it's the current-pass remaining time, plus the remaining time for the additional passes. Luckily (for our analysis), if we know the current pass fails, the remaining time for the additional passes is distributed exactly the same as the apriori remaining time. 

Let $F$ be the event that the current pass is successful, and denote $z=\Pr[F]$. To compute $z$, note that the number of good labels in the remaining labels is also binomially distributed, and that the success of each nonce is completely indpendent of the others, so the probability that *no* nonce will succeed is the product of the individual probabilities.




In [35]:
# This computes `z` from above
def pass_success_prob_midpass(gamma: float, goodlabels: list[int], k1: int, k2: int, m: int, n: int) -> float:
    '''Computes the probability that the current pass is successful

    :param float     gamma: fraction of labels processed so far
    :param list[int] goodlabels: number of good labels found for each nonce (should be of length m)
    '''
    nprime = floor((1-gamma) * n)  # Number of labels remaining in pass
    unsuccessprob = 1
    for numgood in goodlabels:
        k2prime = k2 - numgood # Number of labels left to find for this nonce.
        qprime = binom.sf(k2prime-1, nprime, k1/n) # Probability that this nonce is successful in this pass.
        unsuccessprob *= (1-qprime) # The probability that none of the nonces is successful is the product of individual probabilities 
    return 1 - unsuccessprob
    



#### Expected remaining duration
Following the analysis above, the expected remaining time assuming that $r'$ k2pows remain for this pass, and we read $\gamma\cdot n$ of the labels, is 
$$ 
  \begin{align*}
  E[\text{remaining time}]  &= \Pr[F]\cdot E[T|F]+(1-\Pr[F])\cdot E[T|\neg F] \\
                            &= z\cdot E[T']+(1-z)\cdot (E[T']+E[T]) \\
                            &= E[T']+(1-z)\cdot E[T] \\
                            &= r'\cdot d/\text{hashrate}+(1-\gamma)\cdot R +(1-z)\cdot E[T]
  \end{align*}
$$
where $E[T]$ is the expected apriori duration.



In [36]:

def expected_total_time_midpass(rprime: int, gamma: float, goodlabels: list[int], R: float, d: int, hashrate: int, r: int, k1: int, k2: int, m: int, n: int) -> float:
    z = pass_success_prob_midpass(gamma, goodlabels, k1, k2, m, n) # Probability of current pass success
    qm = pass_success_prob(k1, k2, m, n) # A-priori probability of pass success

    return  expected_pass_k2pow_time(d, hashrate, rprime)+ (1-gamma)*R + (1-z)*expected_total_time_apriori(R, d, hashrate, r, qm)


### Percentile remaining duration.

To compute the $p^*$-percentile --- that is, the total time $T^*$ such that we will succeed within time $T^*$ with probability at least $p^*$, we again have to consider subcases:

1. We're still in the k2pow phase ($r'>0$). In this case, the percentile calculation is exactly the same as the apriori calculation, except the total number of k2pows in the first round is smaller.
2. We're in the read/process phase ($r'=0$). In this case, we consider two subcases:

    1. $\Pr[F]\ge p^*$: i.e., the probability that we succeed in the current pass is at least $p^*$. In this case, the time $T^*$ is just the time remaining in the current pass: $(1-\gamma)\cdot R$

    2. $\Pr[F] < p^*$: in this case let $\hat T$ be the total time until success.
   
       Then $\Pr[\hat T>T^*] = (1-z)\cdot \Pr[\hat T>T^* | \neg F] + z\cdot \Pr[\hat T>T^* | F]$. However, since we know that $T^*$ must be greater than the remaining time in the current pass (otherwise we would be in case (a)), $\Pr[\hat T>T^* | F] = 0$. Thus, $\Pr[\hat T>T^*] = (1-z)\cdot \Pr[\hat T>T^* | \neg F]$. Conditioned on the current pass failing, we know that we will require at least the current pass, plus $T$ additional time, where $T$ is distributed according to the apriori distribution above (without conditioning).
   
       Hence
       $$ \Pr[\hat T>T^*] = (1-z)\cdot \Pr[T > T^* - (1-\gamma)\cdot R] \ . $$

       To find $T^*$ such that  $\Pr[\hat T>T^*] \le 1-p^*$, we must find $T^\dagger=T^*-(1-\gamma)\cdot R$ such that $Pr[T > T^\dagger] \le \frac{1-p^*}{1-z}$; that is, we take the $\left(1-\frac{1-p^*}{1-z}\right)$-percentile apriori duration, and add $(1-\gamma)\cdot R$.


In [37]:
def percentile_total_time_midpass(pstar: float, rprime: int, gamma: float, goodlabels: list[int], R: float, d: int, hashrate: int, r: int, k1: int, k2: int, m: int, n: int) -> float:
    qm = pass_success_prob(k1, k2, m, n) # A-priori probability of pass success
    if rprime > 0:
        # We're in the k2pow phase (case 1)
        return percentile_opt_total_time_apriori(pstar, R, d, hashrate, r, qm, rprime)

    # Were in case (2)        
    z = pass_success_prob_midpass(gamma, goodlabels, k1, k2, m, n) # Probability of current pass success

    if z >= pstar:
        # case 2(a)
        return (1-gamma)*R
        
    # case 2(b)
    tdagger = percentile_opt_total_time_apriori(1-(1-pstar)/(1-z), R, d, hashrate, r, qm)

    return tdagger + (1-gamma)*R

## Examples

In [44]:
## Protocol parameters

### Hashing-related
nonce_batch_size = 16 # Number of nonces in a "batch" 
unit_size = 256 # unit size in GiB
label_size = 16 # Label size in bytes

labels_per_unit = unit_size * 2**30 / label_size

### Post Params
k1 = 26
k2 = 37

postPowDifficulty = 0x000dfb23b0979b4b  # Per-nonce-batch difficulty for one SU * 2^64?

def k2pow_d(n: int):
    """ Difficulty for a single nonce batch and n total labels """
    return 2**64 * n/(labels_per_unit * postPowDifficulty)

dm(f"""
### Global parameters
* $k_1={k1}$
* $k_2={k2}$
* $b = {nonce_batch_size}$ (Number of nonces in a k2pow batch).
* $SU = 2^{{ {log(labels_per_unit,2):.1f} }}$ labels.
* $d = 2^{{ {log(k2pow_d(labels_per_unit),2):.1f} }}$ (expected k2pow hashes per nonce batch for one SU).
""")

## Node-specific parameters
class PoSTer:
    def __init__(self, name, cpu_aes_MiB_sec_for_nonce_batch, read_mb_per_sec, randomx_per_sec):
        self.name = name
        self.cpu_aes_MiB_sec_for_nonce_batch = cpu_aes_MiB_sec_for_nonce_batch
        self.read_mb_per_sec = read_mb_per_sec
        self.randomx_per_sec = randomx_per_sec # single-threaded k2pow performance
        
        # Computed values
        self.nonce_labels_sec = (self.cpu_aes_MiB_sec_for_nonce_batch * 2**20 * nonce_batch_size) / label_size
        self.read_labels_sec = self.read_mb_per_sec * 2**20 / label_size

    def label_process_time(self, n, m):  # R from the analysis above
        ''' Time to process n labels in seconds'''
        return n/min(self.nonce_labels_sec / m,  self.read_labels_sec) 

    def expected_k2pow_time(self, n, r, threads=1): # expected total time for r k2pow solutions
        return expected_pass_k2pow_time(k2pow_d(n), self.randomx_per_sec * threads, r)
    
    # Returns expected total remaining time
    def expected_remaining_time(self, k2pows_found, labels_processed, goodlabels, n, m, threads=1): 
        R = self.label_process_time(n, m)
        hashrate = self.randomx_per_sec * threads
        gamma = labels_processed / n
        d = k2pow_d(n)
        r = m / nonce_batch_size
        
        return expected_total_time_midpass(r - k2pows_found, gamma, goodlabels, R, d, hashrate, r, k1, k2, m, n)

    def percentile_k2pow_time(self, pstar, n, r, threads=1): # expected total time for r k2pow solutions
        return percentile_k2pow_time(pstar, k2pow_d(n), self.randomx_per_sec * threads, r)
    
    def percentile_remaining_time(self, pstar, k2pows_found, labels_processed, goodlabels, n, m, threads=1):  
        R = self.label_process_time(n, m)
        hashrate = self.randomx_per_sec * threads
        gamma = labels_processed / n
        d = k2pow_d(n)
        r = m / nonce_batch_size
                
        return percentile_total_time_midpass(pstar, r - k2pows_found, gamma, goodlabels, R, d, hashrate, r, k1, k2, m, n)

posters = {
   'fast': PoSTer("i7-12700k/slow SSD", cpu_aes_MiB_sec_for_nonce_batch=2028, read_mb_per_sec=200, randomx_per_sec=814.71),
   'slow': PoSTer("Intel(R) Pentium(R) Silver J5040 CPU @ 2.00GHz/fast HDD", cpu_aes_MiB_sec_for_nonce_batch=663, read_mb_per_sec = 150, randomx_per_sec=272.16),
}


### Global parameters
* $k_1=26$
* $k_2=37$
* $b = 16$ (Number of nonces in a k2pow batch).
* $SU = 2^{ 34.0 }$ labels.
* $d = 2^{ 12.2 }$ (expected k2pow hashes per nonce batch for one SU).


In [68]:
# Number of units
su = 4
m = 64
r = m/nonce_batch_size

# Number of labels
n = labels_per_unit * su

qm = pass_success_prob(k1,k2,m,n)
   
poster = posters['fast']

dm(f'''
### Sample node parameters
Using {su} units of space, i.e., $n=2^{{ {log(n,2)} }}$ and $m={m}$ parallel nonces ($r={r}$ nonce batches per pass)

A single pass succeeds w.p. $2^{{ {log(qm,2):.2f} }}$

A(n) {poster.name} can:
* Perform $2^{{ {log(poster.randomx_per_sec,2):.1f} }}$ k2pow hashes per second (single-threaded)
* Hash $2^{{ {log(poster.nonce_labels_sec,2):.1f} }}$ labels per second (for a single nonce) and
* Read $2^{{ {log(poster.read_labels_sec,2):.1f} }}$ labels per second.

## Apriori durations

### Expected time
* Expected total time: $E[T]={poster.expected_remaining_time(0, 0, [], n, m, threads=1)/60:.1f}$ minutes.
* The expected number of passes is ${1/qm:.2f}$
* Expected k2pow time: ${poster.expected_k2pow_time(n, r)/60:.1f}$ minutes per pass, ${poster.expected_k2pow_time(n, r/qm)/60:.1f}$ minutes total

### $75^{{th}}$-percentile
* For $T^* = {poster.percentile_remaining_time(0.75,0, 0, [], n, m, threads=1)/60:.1f}$ minutes, $\Pr[T > T^*] < 0.75$.
* The number of passes is at most ${percentile_passes(0.75,qm)}$ with probability 0.75.
* k2pow time exceeds ${poster.percentile_k2pow_time(0.75, n, r)/60:.1f}$ minutes per pass with less than 25% probability


### $99^{{th}}$-percentile
* For $T^* = {poster.percentile_remaining_time(0.99,0, 0, [], n, m, threads=1)/60:.1f}$ minutes, $\Pr[T > T^*] < 0.99$.
* The number of passes is at most ${percentile_passes(0.99,qm)}$ with probability 0.99.
* k2pow time exceeds ${poster.percentile_k2pow_time(0.99, n, r)/60:.1f}$ minutes per pass with less than 1% probability

''')




### Sample node parameters
Using 4 units of space, i.e., $n=2^{ 36.0 }$ and $m=64$ parallel nonces ($r=4.0$ nonce batches per pass)

A single pass succeeds w.p. $2^{ -0.33 }$

A(n) i7-12700k/slow SSD can:
* Perform $2^{ 9.7 }$ k2pow hashes per second (single-threaded)
* Hash $2^{ 31.0 }$ labels per second (for a single nonce) and
* Read $2^{ 23.6 }$ labels per second.

## Apriori durations

### Expected time
* Expected total time: $E[T]=200.9$ minutes.
* The expected number of passes is $1.26$
* Expected k2pow time: $1.5$ minutes per pass, $1.9$ minutes total

### $75^{th}$-percentile
* For $T^* = 179.0$ minutes, $\Pr[T > T^*] < 0.75$.
* The number of passes is at most $1$ with probability 0.75.
* k2pow time exceeds $2.4$ minutes per pass with less than 25% probability


### $99^{th}$-percentile
* For $T^* = 360.4$ minutes, $\Pr[T > T^*] < 0.99$.
* The number of passes is at most $3$ with probability 0.99.
* k2pow time exceeds $4.5$ minutes per pass with less than 1% probability



In [65]:
dm(f'''
## Mid-pass, found $r'={ceil(r/4)}$ of ${floor(r)}$ k2pows
* Expected k2pow time remaining in this pass: ${poster.expected_k2pow_time(n, r-ceil(r/4))/60:.1f}$ minutes
* Total remaining time is less than ${poster.percentile_remaining_time(0.75,ceil(r/4), 0, [], n, m, threads=1)/60:.1f}$ minutes with 75% probability.
* Total remaining time is less than ${poster.percentile_remaining_time(0.99,ceil(r/4), 0, [], n, m, threads=1)/60:.1f}$ minutes with 99% probability.
''')


## Mid-pass, found $r'=1$ of $4$ k2pows
* Expected k2pow time remaining in this pass: $1.2$ minutes
* Total remaining time is less than $178.6$ minutes with 75% probability.
* Total remaining time is less than $359.9$ minutes with 99% probability.


In [88]:
# Find a random "midpass" setup.

nprocessed = random.randint(1,n) # Uniformly distributed in the middle
goodlabels = binom.rvs(nprocessed, k1/n, size=m) # m binomially distributed values (number of good labels found so far for each nonce)
gamma = nprocessed / n

dm(f'''
## Mid-pass, post-k2pow, read/processed ${nprocessed}$ out of ${ceil(n)}$ ($\gamma={gamma:.2f}$)
* goodlabels={goodlabels}
* Remaining pass time: ${poster.label_process_time(n-nprocessed, m)/60:.1f}$ out of ${poster.label_process_time(n, m)/60:.1f}$ minutes.
* Expected total time remaining: ${poster.expected_remaining_time(r, nprocessed, goodlabels, n, m)/60:.1f}$ minutes.
* Tme remaining is less than ${poster.percentile_remaining_time(0.75,r, nprocessed, goodlabels, n, m)/60:.1f}$ minutes with 75% probability.
* Tme remaining is less than ${poster.percentile_remaining_time(0.99,r, nprocessed, goodlabels, n, m)/60:.1f}$ minutes with 99% probability.
''')


## Mid-pass, post-k2pow, read/processed $46870560306$ out of $68719476736$ ($\gamma=0.68$)
* goodlabels=[21 24 20 20 14 22 15 28 19 16 16 10 18 20 21 22 22 15 18 14 12 13 15 16
 16 16 13 16 13 24 18  9 14 12 17 26 15 17 14 15 10 19 13 17 11 17 13 14
 14  9 17 17 19 13 15 19 21 24 18 19 18  9 20 21]
* Remaining pass time: $27.8$ out of $87.4$ minutes.
* Expected total time remaining: $60.9$ minutes.
* Tme remaining is less than $116.3$ minutes with 75% probability.
* Tme remaining is less than $298.0$ minutes with 99% probability.
